# Mejores matrices Origen-Destino

En el primer notebook de accesibiliodad implementamos nuestros modelos utilizando la distancia euclidiana. Esto es, desde luego, relativamente sencillo de calcular, sin embargo claramente no es lo ideal. El siguiente paso lógico es utilizar la conectividad sobre la red de calles.

Hay muchísimas formas de obtener matrices de costo utilizando la red de calles, desde luego todas ellas van a tener algunas complicaciones, se necesitan más datos (la red de calles, dah) y es computacionalmente intensivo. En la documentación de access tenemos una [lista](https://access.readthedocs.io/en/latest/resources.html) de diferentes herramientas para crear estas matrices.

Nosotros vamos a usar en este ejemplo [OSRM](http://project-osrm.org/) que es un motor de rutas que utiliza datos de OSM. Para esto tenemos una instalación de OSRM en un servidor de CentroGeo. Para acceder a ese servidor tienen que sacar una cuenta con su correo institucional en [tailscale](tailscale.com). Ya que tengan su cuenta necesitan seguir las [instrucciones](https://tailscale.com/download/windows) para instalar el cliente de tailscale en su plataforma. Ya con el cliente configurado pueden acceder a la interfaz gráfica aquí:

http://100.67.66.19:9966/

En lo que sigue de este notebook voy a usar datalab en lugar de la ip para escribir menos (para que eso funcione necesitan editar el hosts file de sus compus o el equivalente en windows).

OSRM es un servidor que nos provee un API para hacer algunos cálculos de rutas. La documentación completa del API la pueden ver [aquí](http://project-osrm.org/docs/v5.24.0/api/#). Para hacer peticiones al API desde Python, vamos a usar la librería [requests](https://docs.python-requests.org/en/master/). Para empezar, hagamos una petición pidiendo la ruta y las instrucciones entre dos puntos. 

## Ejemplo de ruta

In [2]:
import requests
r = requests.get('http://datalab:5000/route/v1/driving/-99.14428918256026,19.351424122115958;-99.16261403345554,19.35211246418011?steps=true')
r.json()

{'code': 'Ok',
 'routes': [{'geometry': 'oqbuBjcc|Q|Fn@VqB`IhA?zCyEh@CSg@@FhBM??[_@@aCtBeEl]bEv]_AV_EbCmAb@\\hCmDf@vDb\\',
   'legs': [{'steps': [{'intersections': [{'out': 0,
         'entry': [True],
         'bearings': [192],
         'location': [-99.144376, 19.351441]},
        {'out': 2,
         'location': [-99.144546, 19.350626],
         'bearings': [15, 90, 195, 270],
         'entry': [False, True, True, True],
         'in': 0},
        {'out': 2,
         'location': [-99.144595, 19.350288],
         'bearings': [15, 105, 195, 285],
         'entry': [False, True, True, True],
         'in': 0}],
       'driving_side': 'right',
       'geometry': 'oqbuBjcc|QRBlC\\bAHVB',
       'mode': 'walking',
       'duration': 103.7,
       'maneuver': {'bearing_after': 192,
        'type': 'depart',
        'modifier': 'left',
        'bearing_before': 0,
        'location': [-99.144376, 19.351441]},
       'weight': 103.7,
       'distance': 143.9,
       'name': 'Calle Tenis'},
 

Como ven, es muy fácil de usar, desde luego la salida es un poco complicada, pero en el fondo es sólo un json, entonces es sencillo de parsear y utilizar como mejor nos convenga. 


## Ejemplo de matriz

Como nuestro problema básico ahorita no es obtener rutas sino calcular matrices origen destino, nos vamos a concentrar en ello en lugar de dar un paseo completo por el API de OSRM.

El único servicio que vamos a necesitar el [table-service](http://project-osrm.org/docs/v5.24.0/api/#table-service) que toma dos listas de coordenadas y nos regresa las distancias (sobre la red) y costos entre todos los pares de coordenadas. Justo lo que queremos.

Para empezar a trabajar con nuestros datos, leamos los datos de centroides de áreas verdes y agebs.

In [3]:
import pandas as pd
agebs_zmvm_centroides = pd.read_pickle("datos/agebs_zmvm_centroides.pkl")
areas_verdes_centroides = pd.read_pickle("datos/areas_verdes_centroides.pkl")

Tomemos sólo los primeros registros de cada una, ahorita sólo queremos entender la estructura que nos regresa el API y cómo crear la matriz que queremos

In [4]:
# Tomamos los primeros 5 registros de cada dataframe
origenes = agebs_zmvm_centroides.head()
destinos = areas_verdes_centroides.head()

Ahora tenemos que manufacturar el string de la petición al API. Viendo la documentación, el servicio nos regresa la distancia (o el costo) entre todos los pares de coordenadas, o bien, podemos identificar un par de coordenadas como el origen y entonces nos regrasa la distancia desde ese punto a todos los demás. Por la estructura de nuestros datos, lo más fácil es tomar el primer punto de origen y calcular las distancias a todos los demás y después iterar sobre los origenes.


Una petición del tipo que queremos se ve así:

````bash
curl 'http://router.project-osrm.org/table/v1/driving/13.388860,52.517037;13.397634,52.529407;13.428555,52.523219?sources=0'
````

El primer par de coordenadas es el origen y el resto son los destinos. 


In [5]:
# Hacemos una lista de las latitudes y longitudes
ys = destinos.geometry.to_crs(4326).y.to_list()
xs = destinos.geometry.to_crs(4326).x.to_list()
# zipeamos la lista
pares = list(zip(ys,xs))
pares

[(19.629064345383593, -98.91512568994419),
 (19.660166196043452, -98.90433315071964),
 (19.625788572945044, -98.88990441181032),
 (19.625562387164596, -98.88858293729425),
 (19.628150142795214, -98.89124996016245)]

transformamos la lista en el string que necesitamos

In [11]:
s = []
for p in pares:
    a = str(p[1]) + "," + str(p[0])
    s.append(a)
destinos_str = ";".join(s)
destinos_str

'-98.91512568994419,19.629064345383593;-98.90433315071964,19.660166196043452;-98.88990441181032,19.625788572945044;-98.88858293729425,19.625562387164596;-98.89124996016245,19.628150142795214'

Hacemos el string del origen

In [10]:
origen_str = str(origenes.to_crs(4326).iloc[0].geometry.x) + ',' + str(origenes.to_crs(4326).iloc[0].geometry.y)
origen_str

'-99.26028762490587,19.32199032738617'

Hacemos el string completo del request al API

In [12]:
base_url = 'http://router.project-osrm.org/table/v1/driving/'
full_request = base_url + origen_str + ';' + destinos_str + '?sources=0'
full_request

'http://router.project-osrm.org/table/v1/driving/-99.26028762490587,19.32199032738617;-98.91512568994419,19.629064345383593;-98.90433315071964,19.660166196043452;-98.88990441181032,19.625788572945044;-98.88858293729425,19.625562387164596;-98.89124996016245,19.628150142795214?sources=0'

Ahora podemos hacer la petición

In [ ]:
r = requests.get(full_request)
r.json()

{'code': 'Ok',
 'durations': [[0, 3064.7, 3463.5, 3389.7, 3404.2, 3346.1]],
 'sources': [{'hint': 'G2pBiXNqQYkRAAAADwAAAAAAAAAAAAAAorFJQWKOHkEAAAAAAAAAABEAAAAPAAAAAAAAAAAAAACVRAAAamgV-mrUJgGAaBX6htQmAQAALxOIUjEx',
   'distance': 3.86663,
   'location': [-99.26031, 19.321962],
   'name': 'Calle Orquídea'}],
 'destinations': [{'hint': 'G2pBiXNqQYkRAAAADwAAAAAAAAAAAAAAorFJQWKOHkEAAAAAAAAAABEAAAAPAAAAAAAAAAAAAACVRAAAamgV-mrUJgGAaBX6htQmAQAALxOIUjEx',
   'distance': 3.86663,
   'location': [-99.26031, 19.321962],
   'name': 'Calle Orquídea'},
  {'hint': 'vfA1if___38HAAAADAAAADMAAAA4AAAAiBoMQQqBwEAZ0HpCvtOKQgcAAAAMAAAAMwAAADgAAACVRAAAjawa-kOEKwHKrBr6CIQrAQQAHw6IUjEx',
   'distance': 9.143012,
   'location': [-98.915187, 19.629123],
   'name': ''},
  {'hint': '9fM1iffzNYkoAAAAPQAAABMAAAAfAAAAgX_eQcd-J0IsqVFBD8OnQSgAAAA9AAAAEwAAAB8AAACVRAAAPNca-m_9KwHz1hr6hv0rAQEAbwyIUjEx',
   'distance': 8.067194,
   'location': [-98.90426, 19.660143],
   'name': 'Calle Río San Juan'},
  {'hint': 'La41iTauNYk

De acuerdo a la documentación de OSRM, lo que nos regresa el API es un json que en la entrada `durations` tiene las duraciones de los viajes a cada uno de los destinos conservando el índice en el que los mandamos, las demás entradas contienen información adicional que en este caso no vamos a utilizar. Entonces, lo que necesitamos hacer es simplemete extraer la lista de durations

In [15]:
r.json()['durations'][0]

[0, 3064.7, 3463.5, 3389.7, 3404.2, 3346.1]

Ahora necesitamos estructurar los datos de forma que podamos obtener el dataframe de costos como lo necesitamos para `access`